<h3> MPC modeling </h3>
$\begin{align*}
State\;Space : z &= [x, y, v,\phi]\\
Action\;Space : u &= [a, \delta]\\
x&: x-position, y:y-position, v:velocity, φ: yaw angle\\
a&: accellation, δ: steering angle\\
\end{align*}$

<h3>Cost:</h3>

$\begin{align*}
min\ Q_f(z_{T,ref}-z_{T})^2+Q\Sigma({z_{t,ref}-z_{t}})^2+R\Sigma{u_t}^2+R_d\Sigma({u_{t+1}-u_{t}})^2
\end{align*}$

z_ref : target states

<h3>Constraints :</h3>

$\begin{align*}
z_{t+1}=Az_t+Bu+C\\
Maximum\;steering\; speed &=|u_{t+1}-u_{t}|<du_{max}\\
Maximum\;steering\;angle &=|u_{t}|<u_{max}\\
Initial\;state &= z_0 = z_{0,ob}\\
Maximum\;and\;minimum \;speed &= v_{min} < v_t < v_{max}\\
Maximum\;and minimum\;input &=u_{min} < u_t < u_{max}\\
\end{align*}$



### Car Steering Model with Linearization




$\begin{align*}
 \dot{x} &= vcos(\phi)                      &\\
 \dot{y} &= vsin(\phi)       & \dot{z} &=\frac{\partial }{\partial z} z = f(z, u) = A'z+B'u\\
 \dot{v} &= a                               &\\
 \dot{\phi} &= \frac{vtan(\delta)}{L}       &\\
\end{align*}$




where

$\begin{equation*}
A' =
\begin{bmatrix}
\frac{\partial }{\partial x}vcos(\phi) & 
\frac{\partial }{\partial y}vcos(\phi) & 
\frac{\partial }{\partial v}vcos(\phi) &
\frac{\partial }{\partial \phi}vcos(\phi)\\
\frac{\partial }{\partial x}vsin(\phi) & 
\frac{\partial }{\partial y}vsin(\phi) & 
\frac{\partial }{\partial v}vsin(\phi) &
\frac{\partial }{\partial \phi}vsin(\phi)\\
\frac{\partial }{\partial x}a& 
\frac{\partial }{\partial y}a& 
\frac{\partial }{\partial v}a&
\frac{\partial }{\partial \phi}a\\
\frac{\partial }{\partial x}\frac{vtan(\delta)}{L}& 
\frac{\partial }{\partial y}\frac{vtan(\delta)}{L}& 
\frac{\partial }{\partial v}\frac{vtan(\delta)}{L}&
\frac{\partial }{\partial \phi}\frac{vtan(\delta)}{L}\\
\end{bmatrix}
　=
\begin{bmatrix}
0 & 0 & cos(\bar{\phi}) & -\bar{v}sin(\bar{\phi})\\
0 & 0 & sin(\bar{\phi}) & \bar{v}cos(\bar{\phi}) \\
0 & 0 & 0 & 0 \\
0 & 0 &\frac{tan(\bar{\delta})}{L} & 0 \\
\end{bmatrix}
\end{equation*}$



$\begin{equation*}
B' =
\begin{bmatrix}
\frac{\partial }{\partial a}vcos(\phi) &
\frac{\partial }{\partial \delta}vcos(\phi)\\
\frac{\partial }{\partial a}vsin(\phi) &
\frac{\partial }{\partial \delta}vsin(\phi)\\
\frac{\partial }{\partial a}a &
\frac{\partial }{\partial \delta}a\\
\frac{\partial }{\partial a}\frac{vtan(\delta)}{L} &
\frac{\partial }{\partial \delta}\frac{vtan(\delta)}{L}\\
\end{bmatrix}
　=
\begin{bmatrix}
0 & 0 \\
0 & 0 \\
1 & 0 \\
0 & \frac{\bar{v}}{Lcos^2(\bar{\delta})} \\
\end{bmatrix}
\end{equation*}$



Forward Euler Discretization with sampling time dt.
Using first degree Tayer expantion around zbar and ubar

$\begin{align*}
z_{k+1}&=z_k+f(z_k,u_k)dt\\
z_{k+1}&=z_k+(f(\bar{z},\bar{u})+A'z_k+B'u_k-A'\bar{z}-B'\bar{u})d\\
z_{k+1}&=(I + dtA')z_k+(dtB')u_k + (f(\bar{z},\bar{u})-A'\bar{z}-B'\bar{u})dt\\
z_{k+1}&=Az_k+Bu_k +C\\
\end{align*}$


So,

$\begin{equation*}
A = (I + dtA')
=
\begin{bmatrix} 
1 & 0 & cos(\bar{\phi})dt & -\bar{v}sin(\bar{\phi})dt\\
0 & 1 & sin(\bar{\phi})dt & \bar{v}cos(\bar{\phi})dt \\
0 & 0 & 1 & 0 \\
0 & 0 &\frac{tan(\bar{\delta})}{L}dt & 1 \\
\end{bmatrix}
\end{equation*}$
$\begin{equation*}
B = dtB'
=
\begin{bmatrix} 
0 & 0 \\
0 & 0 \\
dt & 0 \\
0 & \frac{\bar{v}}{Lcos^2(\bar{\delta})}dt \\
\end{bmatrix}
\end{equation*}$

$\begin{equation*}
C = (f(\bar{z},\bar{u})-A'\bar{z}-B'\bar{u})dt\\
= dt(
\begin{bmatrix} 
\bar{v}cos(\bar{\phi})\\
\bar{v}sin(\bar{\phi}) \\
\bar{a}\\
\frac{\bar{v}tan(\bar{\delta})}{L}\\
\end{bmatrix}
-
\begin{bmatrix} 
\bar{v}cos(\bar{\phi})-\bar{v}sin(\bar{\phi})\bar{\phi}\\
\bar{v}sin(\bar{\phi})+\bar{v}cos(\bar{\phi})\bar{\phi}\\
0\\
\frac{\bar{v}tan(\bar{\delta})}{L}\\
\end{bmatrix}
-
\begin{bmatrix} 
0\\
0 \\
\bar{a}\\
\frac{\bar{v}\bar{\delta}}{Lcos^2(\bar{\delta})}\\
\end{bmatrix}
)\\
=
\begin{bmatrix} 
\bar{v}sin(\bar{\phi})\bar{\phi}dt\\
-\bar{v}cos(\bar{\phi})\bar{\phi}dt\\
0\\
-\frac{\bar{v}\bar{\delta}}{Lcos^2(\bar{\delta})}dt\\
\end{bmatrix}
\end{equation*}$

Inspired from - 

[AtsushiSakai/PythonRobotics](https://github.com/AtsushiSakai/PythonRobotics/blob/f51a73f47cb922a12659f8ce2d544c347a2a8156/PathTracking/model_predictive_speed_and_steer_control/model_predictive_speed_and_steer_control.py#L247-L301)

<h3> Reference </h3>

- [AtsushiSakai/PythonRobotics](https://github.com/AtsushiSakai/PythonRobotics/blob/eb6d1cbe6fc90c7be9210bf153b3a04f177cc138/PathTracking/model_predictive_speed_and_steer_control/model_predictive_speed_and_steer_control.py#L80-L102)
- [Welcome to CVXPY 1\.0 — CVXPY 1\.0\.6 documentation](http://www.cvxpy.org/)
- [Vehicle Dynamics and Control \| Rajesh Rajamani \| Springer](http://www.springer.com/us/book/9781461414322)

- [MPC Course Material \- MPC Lab @ UC\-Berkeley](http://www.mpc.berkeley.edu/mpc-course-material)





<h3> MPC modeling </h3>
$\begin{align*}
State\;Space : z &= [x, y, v,\phi]\\
Action\;Space : u &= [a, \delta]\\
x&: x-position, y:y-position, v:velocity, φ: yaw angle\\
a&: accellation, δ: steering angle\\
\end{align*}$
<h3>State Space model for Car system</h3>

$\begin{equation*}
A = 
\begin{bmatrix} 
1 & 0 & cos(\bar{\phi})dt & -\bar{v}sin(\bar{\phi})dt\\
0 & 1 & sin(\bar{\phi})dt & \bar{v}cos(\bar{\phi})dt \\
0 & 0 & 1 & 0 \\
0 & 0 &\frac{tan(\bar{\delta})}{L}dt & 1 \\
\end{bmatrix}
\end{equation*}$
$\begin{equation*}
B =
\begin{bmatrix} 
0 & 0 \\
0 & 0 \\
dt & 0 \\
0 & \frac{\bar{v}}{Lcos^2(\bar{\delta})}dt \\
\end{bmatrix}
\end{equation*}$

$\begin{align*}
z_{t+1}=Az_t+Bu+C\\
\end{align*}$
<h3>Cost:</h3>

$\begin{align*}
min\ Q_f(z_{T,ref}-z_{T})^2+Q\Sigma({z_{t,ref}-z_{t}})^2+R\Sigma{u_t}^2+R_d\Sigma({u_{t+1}-u_{t}})^2
\end{align*}$

z_ref : target states

<h3>Constraints :</h3>

$\begin{align*}
z_{t+1}=Az_t+Bu+C\\
Maximum\;steering\; speed &=|u_{t+1}-u_{t}|<du_{max}\\
Maximum\;steering\;angle &=|u_{t}|<u_{max}\\
Initial\;state &= z_0 = z_{0,ob}\\
Maximum\;and\;minimum \;speed &= v_{min} < v_t < v_{max}\\
Maximum\;and minimum\;input &=u_{min} < u_t < u_{max}\\
\end{align*}$



<h3>Iterative Linear Quadratic Regulator :</h3>

$\begin{align*}
A.\;&Getting\;Qs[i]\;from\;loss\;and\;Vs[i]:\\
Q_x &= l_x(t) + f_x(t).T, V_x \\
Q_u &= l_u(t) + f_u(t).T, V_x \\
Q_{xx} &= l_{xx}(t) + f_x(t).T * (V_xx * f_x(t) )  \\
Q_{ux} &= l_{ux}(t) + f_u(t).T * (V_xx * f_x(t) )) \\
Q_{uu} &= l_{uu}(t) + (f_u(t).T *(V_xx * f_u(t) ) \\
\end{align*}$
$\begin{align*}
&B.\;Getting\;Ks[i]\;from\;Qs[i]:\\
&Q_{uu_evals}, Q_{uu_evecs} = np.linalg.eig(Q_uu)\\
&Q_{uu_evals}[Q_{uu_evals} < 0] = 0.0\\
&Q_{uu_inv} = Q_{uu_evecs} * (np.diag(1.0/Q_uu_evals), Q_uu_evecs.T)\\              
&k(t) = -1. * (Q_{uu_inv} * Q_u)\\
&K(t) = -1. * (Q_{uu_inv} * Q_ux)\\
\end{align*}$
$\begin{align*}
&C.\;Getting\;V[i+1]\;from\;Ks[i]:\\
V_x &= Q_x - (K[t].T * (Q_uu * k[t]))\\
V_{xx} &= Q_xx - (K[t].T * (Q_uu * K[t]))
\end{align*}$